# Autogluon

Use the Autogluon AutoML library to predict ratings.

In [7]:
# Packages
import pandas as pd
from autogluon.tabular import TabularDataset, TabularPredictor
import os

## Load Data

In [8]:
# list of files in '../../Data/All_Data/All_Data_Fixed_Quarter_Dates'
file_list = [f for f in os.listdir(r'../../Data/All_Data/All_Data_Fixed_Quarter_Dates') if f.endswith('.parquet')]
# read in all parquet files
df = pd.concat([pd.read_parquet(r'../../Data/All_Data/All_Data_Fixed_Quarter_Dates/' + f) for f in file_list])
print('dataframe')
print(df)

dataframe
    ticker fixed_quarter_date earnings_call_date Rating  \
0     AAPL         2014-07-01         2014-04-23     AA   
1     AAPL         2014-10-01         2014-07-22     AA   
2     AAPL         2015-01-01         2014-10-20     AA   
3     AAPL         2015-04-01         2015-01-27     AA   
4     AAPL         2015-07-01         2015-04-27     AA   
..     ...                ...                ...    ...   
912    ZTS         2015-10-01         2015-08-04    BBB   
913    ZTS         2016-01-01         2015-11-03    BBB   
914    ZTS         2016-04-01         2016-02-16    BBB   
915    ZTS         2016-07-01         2016-05-04    BBB   
916    ZTS         2016-10-01         2016-08-03    BBB   

                     Rating Agency Name rating_date      CR_source  \
0    Standard & Poor's Ratings Services  2014-05-27  Supplementary   
1    Standard & Poor's Ratings Services  2014-05-27  Supplementary   
2    Standard & Poor's Ratings Services  2014-05-27  Supplementary   
3

In [9]:
# Print out column names
print('column names')
for col in df.columns:
    print(col)

column names
ticker
fixed_quarter_date
earnings_call_date
Rating
Rating Agency Name
rating_date
CR_source
Rating Rank AAA is 10
Next Rating
Next Rating Date
Previous Rating
Previous Rating Date
next_rating_date_or_end_of_data
credit_rating_year
previous_fixed_quarter_date
days_since_call_on_fixed_quarter
quarter
calls_year
transcript
Calls_source
date
symbol
reportedCurrency
cik
fillingDate
acceptedDate
calendarYear
period
cashAndCashEquivalents
shortTermInvestments
cashAndShortTermInvestments
netReceivables
inventory
otherCurrentAssets
totalCurrentAssets
propertyPlantEquipmentNet
goodwill
intangibleAssets
goodwillAndIntangibleAssets
longTermInvestments
taxAssets
otherNonCurrentAssets
totalNonCurrentAssets
otherAssets
totalAssets
accountPayables
shortTermDebt
taxPayables
deferredRevenue
otherCurrentLiabilities
totalCurrentLiabilities
longTermDebt
deferredRevenueNonCurrent
deferredTaxLiabilitiesNonCurrent
otherNonCurrentLiabilities
totalNonCurrentLiabilities
otherLiabilities
capitalLeas

In [10]:
# Removing columns: 'Rating Rank AAA is 10', 'transcript', 'Investment_Grade', 'Change Direction Since Last Fixed Quarter Date', 'Change Since Last Fixed Quarter Date'
df = df.drop(columns=['Rating Rank AAA is 10', 'transcript', 'Investment_Grade', 'Change Direction Since Last Fixed Quarter Date', 'Change Since Last Fixed Quarter Date'])

## Deploy AutoGluon

In [17]:
# Split into train and test datasets
train_df = df[df['train_test_80_20'] == 'train'].reset_index(drop=True)
test_df = df[df['train_test_80_20'] == 'test'].reset_index(drop=True)

In [18]:
# Convert from pandas to autogluon
train_data = TabularDataset(train_df)
test_data = TabularDataset(test_df)

In [13]:
# Train and Test
predictor = TabularPredictor(label='Rating').fit(train_data=train_data)
predictions = predictor.predict(test_data)

No path specified. Models will be saved in: "AutogluonModels\ag-20240321_054549"
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20240321_054549"
=================== System Info ===================
AutoGluon Version:  1.0.0
Python Version:     3.11.8
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
CPU Count:          8
Memory Avail:       

In [14]:
# Evaluation
predictor.evaluate(test_data, silent=True)

c:\Users\ijyli\anaconda3\envs\capstone\Lib\site-packages\autogluon\tabular\models\fastainn\tabular_nn_fastai.py:200: FutureWarning: The 'downcast' keyword in fillna is deprecated and will be removed in a future version. Use res.infer_objects(copy=False) to infer non-object dtype, or pd.to_numeric with the 'downcast' keyword to downcast numeric results.
  df = df.fillna(column_fills, inplace=False, downcast=False)


{'accuracy': 0.9436117059243397,
 'balanced_accuracy': 0.7462156761693,
 'mcc': 0.9276648341762417}

In [15]:
# Leaderboard of models
predictor.leaderboard(test_data)

c:\Users\ijyli\anaconda3\envs\capstone\Lib\site-packages\autogluon\tabular\models\fastainn\tabular_nn_fastai.py:200: FutureWarning: The 'downcast' keyword in fillna is deprecated and will be removed in a future version. Use res.infer_objects(copy=False) to infer non-object dtype, or pd.to_numeric with the 'downcast' keyword to downcast numeric results.
  df = df.fillna(column_fills, inplace=False, downcast=False)


,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,CatBoost,0.945753,0.930743,accuracy,0.067332,0.088064,473.677427,0.067332,0.088064,473.677427,1,True,8
1,ExtraTreesGini,0.944325,0.929054,accuracy,0.252590,0.300846,2.845058,0.252590,0.300846,2.845058,1,True,9
2,ExtraTreesEntr,0.943612,0.927365,accuracy,0.218510,0.301146,2.968959,0.218510,0.301146,2.968959,1,True,10
3,WeightedEnsemble_L2,0.943612,0.940878,accuracy,0.843436,0.872712,110.868861,0.008587,0.001448,1.584031,2,True,14
4,RandomForestGini,0.940757,0.920608,accuracy,0.147656,0.111785,8.539341,0.147656,0.111785,8.539341,1,True,6
5,LightGBMXT,0.940043,0.935811,accuracy,0.138857,0.075610,27.379339,0.138857,0.075610,27.379339,1,True,4
6,RandomForestEntr,0.939329,0.925676,accuracy,0.182136,0.112037,10.979592,0.182136,0.112037,10.979592,1,True,7
7,LightGBM,0.937901,0.929054,accuracy,0.096814,0.044241,30.076970,0.096814,0.044241,30.076970,1,True,5
8,NeuralNetFastAI,0.936474,0.932432,accuracy,0.133217,0.082578,37.886198,0.133217,0.082578,37.886198,1,True,3
9,XGBoost,0.935760,0.929054,accuracy,0.168913,0.033088,49.525367,0.168913,0.033088,49.525367,1,True,11


In [19]:
# Feature importance
fi = predictor.feature_importance(test_data)
fi

These features in provided data are not utilized by the predictor and will be ignored: ['Rating Agency Name', 'Calls_source', 'symbol', 'reportedCurrency', 'calendarYear', 'period', 'totalLiabilitiesAndTotalEquity', 'cik_cash_flow_statement', 'operatingCashFlow', 'cik_income_statement', 'Company Name', 'train_test_80_20']
Computing feature importance via permutation shuffling for 149 features using 1395 rows with 5 shuffle sets...
c:\Users\ijyli\anaconda3\envs\capstone\Lib\site-packages\autogluon\tabular\models\fastainn\tabular_nn_fastai.py:200: FutureWarning: The 'downcast' keyword in fillna is deprecated and will be removed in a future version. Use res.infer_objects(copy=False) to infer non-object dtype, or pd.to_numeric with the 'downcast' keyword to downcast numeric results.
  df = df.fillna(column_fills, inplace=False, downcast=False)
	1018.31s	= Expected runtime (203.66s per shuffle set)
c:\Users\ijyli\anaconda3\envs\capstone\Lib\site-packages\autogluon\tabular\models\fastainn\ta

,importance,stddev,p_value,n,p99_high,p99_low
rating_on_previous_fixed_quarter_date,0.035986,0.005788,0.000078,5,0.047904,0.024068
Next Rating,0.008172,0.001302,0.000075,5,0.010853,0.005491
rating_on_previous_fixed_quarter_date AAA is 10,0.005018,0.002822,0.008230,5,0.010829,-0.000793
rating_date,0.002007,0.001063,0.006733,5,0.004196,-0.000182
date,0.002007,0.000785,0.002318,5,0.003624,0.000390
...,...,...,...,...,...,...
previous_fixed_quarter_date,-0.001290,0.000935,0.981659,5,0.000634,-0.003215
generalAndAdministrativeExpenses,-0.001290,0.000321,0.999578,5,-0.000630,-0.001950
accumulatedOtherComprehensiveIncomeLoss,-0.001290,0.000321,0.999578,5,-0.000630,-0.001950
netReceivables,-0.001434,0.000000,0.500000,5,-0.001434,-0.001434
